# Flood Adaptation: Minimal Model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as a simplified representation of household adaptation to flood events in a social network context. The model serves as a template for understanding the fundamental mechanics of flood adapdtation with a given social network.

This directory is structured as follows:
 - A Mesa Agent class `Households` is defined in `agents.py`
 - A Mesa Agent class `Government` is defined in `agents.py` (this agent currently does not perform any action)
 - A Mesa Model class `AdaptationModel` is defined in `model.py`
 - This notebook `demo.ipynb` provides a demonstration of the model and its visualization.

Each household agent in the model represents a household with attributes and behaviors that determine its response to flood events. The flood depth attribute assigned to each household is currently random for demonstration purposes. The decision by each household to adapt is also arbitrary. In a more comprehensive model, this could be replaced with more sophisticated decision-making algorithms.

The current implementation includes several simplifications and arbitrary choices, such as random flood depths and basic adaptation logic based on random thresholds. These aspects are designed for clarity and ease of understanding and should be replaced with realistic and sound choices.

You are encouraged to modify, expand, and customize the model. This might include integrating real-world data, implementing more complex decision-making processes for agents, or exploring different network topologies and their effects on social dynamics.

The visualisation provided is basic. Additional aspects can be added for visualization and more in-depth analysis.

_This notebook was tested on 2023-11-13 with Mesa version 2.1.4 on Python 3.12.0._

# Running the model and visualizing the model domain and social network over time

In [13]:
import random

from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx

# Determine the input values for your network and dynamic parameters and how they should be distributed throughout the population.
network_dynamics_dictionary = {
                # The social network structure that is used. Can currently be
                # "erdos_renyi", "barabasi_albert", "watts_strogatz", or "no_network"
                "network": 'watts_strogatz',
                "number_of_households": 50,  # number of household agents
                "probability_of_network_connection": 0.4,  # likeliness of edge being created between two nodes
                "number_of_edges": 3,  # number of edges for BA network
                "number_of_nearest_neighbours": 5,  # number of nearest neighbours for WS social network
                "flood_time_tick": [5],  # list of flood times on the tick
                "flood_severity_probability": (0.5, 1.2),  # Bounds between a Uniform distribution of the flood severity
                "seed": 1  # The seed used to generate pseudo random numbers
            }

# Determine the input values for your agent's attributes and how they should be distributed throughout the population.
attribute_dictionary = {
                "wealth": [0.2, 'UI', (0, 3)],
                "has_child": [0.2, 'B', (0.2)],
                "house_size": [0.05, 'UI', (0, 2)],
                "house_type": [0.05, 'UI', (0, 1)],
                "education_level": [0.05, 'UI', (0, 3)],
                "social_preference": [0.2, 'U', (-1, 1)],
                "age": [0.2, 'N', (33.4, 5)]
            }

# Defining the standard values for agent interaction dynamics in case none are provided.
agent_interaction_dictionary = {
                "household_tolerance": 0.15,  # the tolerance level for each agent
                "bias_change_per_tick": 0.02,  # Bias change per tick when an agent when it's influenced by its network
                "flood_impact_on_bias_factor": 1,  # Defines factor regarding the actual and expected damage of flooding
                "probability_positive_bias_change": 0.5,  # Probability that agent changes it's bias positively
                "probability_negative_bias_change": 0.1,  # Probability that agent changes it's bias negatively
                "adaption_threshold": 0.7  # Threshold of bias an agent needs to adapt
            }

# Initialize the Adaptation Model with 50 household agents.
model = AdaptationModel(network_dynamics_dictionary=network_dynamics_dictionary, attribute_dictionary=attribute_dictionary, agent_interaction_dictionary= agent_interaction_dictionary)
# flood_map_choice can be "harvey", "100yr", or "500yr"

# Calculate positions of nodes for the network plot.
# The spring_layout function positions nodes using a force-directed algorithm,
# which helps visualize the structure of the social network.
pos = nx.spring_layout(model.G)

# Define a function to plot agents on the network.
# This function takes a matplotlib axes object and the model as inputs.
# def plot_network(ax, model):
#     # Clear the current axes.
#     ax.clear()
#     # Determine the color of each node (agent) based on their adaptation status.
#     colors = ['blue' if agent.is_adapted else 'red' for agent in model.schedule.agents]
#     # Draw the network with node colors and labels.
#     nx.draw(model.G, pos, node_color=colors, with_labels=True, ax=ax)
#     # Set the title of the plot with the current step number.
#     ax.set_title(f"Social Network State at Step {model.schedule.steps}", fontsize=12)

# Generate the initial plots at step 0.
# Plot the spatial distribution of agents. This is a function written in the model.py
# model.plot_model_domain_with_agents()

# Plot the initial state of the social network.
# fig, ax = plt.subplots(figsize=(7, 7))
# plot_network(ax, model)
# plt.show()

# Run the model for 20 steps and generate plots every 5 steps.
for step in range(80):
    model.step()
# 
#     # Every 5 steps, generate and display plots for both the spatial distribution and network.
#     # Note the first step is step 0, so the plots will be generated at steps 4, 9, 14, and 19, which are the 5th, 10th, 15th, and 20th steps.
#     if (step + 1) % 10 == 0:
#         # Plot for the spatial map showing agent locations and adaptation status.
#         plt.figure(figsize=(10, 6))
#         model.plot_model_domain_with_agents()
# 
#         # Plot for the social network showing connections and adaptation statuses.
#         fig, ax = plt.subplots(figsize=(7, 7))
#         plot_network(ax, model)
#         plt.show()

step: 0 : 38 adapted with a bias of 0 and a estimated damage of 1!
step: 0 : 1 adapted with a bias of -0.02 and a estimated damage of 1!
step: 0 : 29 adapted with a bias of 0 and a estimated damage of 0.8548709702638162!
step: 0 : 24 adapted with a bias of 0 and a estimated damage of 0.7705434188302884!
step: 0 : 18 adapted with a bias of 0 and a estimated damage of 0.7095551845290607!
step: 0 : 16 adapted with a bias of 0 and a estimated damage of 0.7570011430431214!
step: 0 : 49 adapted with a bias of 0 and a estimated damage of 1!
step: 0 : 12 adapted with a bias of 0 and a estimated damage of 0.7869106585121985!
step: 0 : 17 adapted with a bias of 0 and a estimated damage of 0.9353670355337003!
step: 0 : 41 adapted with a bias of 0 and a estimated damage of 1!
step: 0 : 23 adapted with a bias of 0 and a estimated damage of 0.7065021840590641!
step: 0 : 4 adapted with a bias of 0 and a estimated damage of 0.8309485378482248!
step: 2 : 13 adapted with a bias of 0.02 and a estimated d

In [14]:
agent_data = model.datacollector.get_agent_vars_dataframe()
len([i for i in model.schedule.agents if i.has_child])
agent_data

# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a estimated flood damage of {round(agent.flood_damage_estimated, 3)} and is adapted is {agent.is_adapted} and has identity: {agent.identity}\n')
#     
# for agent in model.schedule.agents:
#     print(f'{agent.unique_id} has an agent bias of {round(agent.general_bias_in_network, 3)}, a impacted bias because of flooding {round(agent.actual_flood_impact_on_bias, 3)}')

    

FloodDepthEstimated  FloodDamageEstimated  FloodDepthActual  \
Step AgentID                                                                
0    0                   0.779350              0.604773          0.000000   
     1                   6.402948              1.000000          0.000000   
     2                   0.642626              0.571093          0.000000   
     3                   0.315795              0.447045          0.000000   
     4                   2.846519              0.830949          0.000000   
...                           ...                   ...               ...   
79   45                  0.536133              0.539459          0.501634   
     46                  1.216463              0.682512          1.448115   
     47                  0.429989              0.500938          0.498942   
     48                  0.182650              0.351448          0.178514   
     49                  9.226017              1.000000          7.411401   

              FloodDamageActual  IsAdapted  identity  FriendsCount  \
Step AgentID                                                         
0    0                 0.000000      False  0.141350             3   
     1                 0.000000      False  0.590319             6   
     2                 0.000000      False  0.360749             2   
     3                 0.000000      False  1.026729             4   
     4                 0.000000      False  1.099769             6   
...                         ...        ...       ...           ...   
79   45                0.527846       True  0.351167             4   
     46                0.712948       True  0.359290             3   
     47                0.526907       True  0.640360             3   
     48                0.347448       True  0.903582             4   
     49                1.000000       True  0.780593             5   

                                                   location  HasChild  \
Step AgentID                                                            
0    0          POINT (253071.6900172176 3316787.418455525)     False   
     1         POINT (267490.4392086614 3294491.7159918016)      True   
     2        POINT (274089.38984379085 3301055.4133319915)     False   
     3         POINT (304709.48301778664 3275641.790944181)     False   
     4          POINT (242972.3748635401 3317007.241808125)     False   
...                                                     ...       ...   
79   45       POINT (219982.05851606763 3313861.5276159765)     False   
     46        POINT (281770.09382179164 3265083.766239415)     False   
     47         POINT (215282.2909264425 3317263.624873443)     False   
     48        POINT (271081.54049757257 3268777.785521252)     False   
     49           POINT (270615.1245270499 3294714.9767383)     False   

              AgentBias  
Step AgentID             
0    0             0.00  
     1             0.00  
     2             0.00  
     3             0.00  
     4             0.00  
...                 ...  
79   45            0.40  
     46            0.50  
     47            0.68  
     48            0.76  
     49            1.22  

[4000 rows x 10 columns]

In [15]:
# sum([i.general_bias_in_network for i in model.schedule.agents])/len(model.schedule.agents)

max([i.actual_flood_impact_on_bias for i in model.schedule.agents])

0.06463296446629974

In [16]:
model_data = model.datacollector.get_model_vars_dataframe()
model_data

,total_adapted_households
0,0
1,12
2,12
3,13
4,14
...,...
75,29
76,29
77,29
78,29
